# TP5

### Implementar el detector de fondo naive usando la mediana como estimador. El algoritmo debe recibir el parámetro N (cantidad de frames utilizados para la estimación) y el intervalo de tiempo para recalcular el fondo.

### Se deben generar las mascaras de foreground y aplicarlas a los frames para segmentar los objetos en movimiento.

### Comparar con alguno de los métodos vistos en la practica basados enmezcla de gaussianas

In [81]:
import numpy as np
import cv2 as cv
from __future__ import print_function
import sys
import os


In [82]:
# Función de detector de fondo naive
def Naive(N,frame,data):
    e1 = cv.getTickCount()  #empiezo a contar
    data.append(frame)
    time=0
    if (len(data) > N):
        naive = np.array(data)
        naive = np.median(naive,axis=0)
        naive = naive.astype(np.uint8)
        cv.imshow('fondo Naive',naive)
        # data.clear()
        #En vez de borrar todo el array solo borramos el primer
        #frame del "array" para calcular la mediana y no perder mas frames
        data.pop(0)
        diff = cv.absdiff(frame_gray,naive)
        ret, thresh = cv.threshold(diff,20,255,cv.THRESH_BINARY)
        e2 = cv.getTickCount()  #Termino de contar
        time = (e2 - e1)/ cv.getTickFrequency()
        #print('Tiempo de calculo Naive',time, end='\r')

        #cargar el array demora 20mseg
        #pero una vez cargado procesar el dato alrededor de 15mseg
        #10 veces mas que el MOG2
    else:
        #Se pierden los primeros N frames antes de calcular la mediana
        thresh = frame

    return thresh,time

In [83]:
metodo = 'MOG2'
# metodo = 'KNN'
if metodo == 'MOG2':
    backSub = cv.createBackgroundSubtractorMOG2()
else:
    backSub = cv.createBackgroundSubtractorKNN()
    backSub = cv.createBackgroundSubtractorKNN(detectShadows = True)

In [84]:
# Abrimos el archivo
#-------------------
filename = 'vtest.avi'
capture = cv.VideoCapture(filename)

if not capture.isOpened:
    print('Falla al abrir el archivo: ' + filename)
    exit(0)



In [85]:
# Corremos la sustraccion
#------------------------
ret, frame = capture.read()
averageValue1 = np.float32(frame)
data=[]
timer =0
fgMask = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
# ****************
# N=cantidad de frames para estimaciòn
N = 20
# *****************

while True:

    # Leemos un frame
    ret, frame = capture.read()
    if frame is None:
        break

    # Aplicamos la sustracción al frame leído "media"

    frame_gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    # N=cantidad de frames para estimaciòn
    # frame_gray = frame para crear el array
    # data = Array para calcular estimaciòn
    fgMask,timer=Naive(N,frame_gray,data)
    #Mog2
    e1_mog = cv.getTickCount()  #empiezo a contar
    mog2=backSub.apply(frame)
    e2_mog = cv.getTickCount()  #empiezo a contar
    print("\r{}".format(f'Tiempo de calculo MOG2: {(e2_mog - e1_mog)/ cv.getTickFrequency()} seg || '), end="")
    print("{}".format(f'Tiempo de calculo Naive: {timer} seg'), end="")
    #print("\r{'Tiempo de calculo Naive'}".format(timer), end="")
    #print('Tiempo de calculo MOG2',(e2_mog - e1_mog)/ cv.getTickFrequency())
    #print('Tiempo de calculo Naive',timer)
    #os.system('clear')
    # Escribimos sobre la imagen el número de frame procesado
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    # Escribimos sobre la imagen el número de frame naive procesado
    cv.rectangle(fgMask, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(fgMask, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    # mostramos frame original e imagen binaria background/foreground
    cv.imshow('Frame', frame)
    cv.imshow('FG Mask con NAIVE', fgMask)
    cv.imshow('FG MOG2', mog2)
    # Corresmos hasta que termine o apriete escape
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break

cv.destroyAllWindows()
capture.release()

Tiempo de calculo MOG2: 0.00679612 seg || Tiempo de calculo Naive: 0.060731701 segg

### Conclusiones


##### Luego de implementar el detector de fondo Naive y comparandolo con el detector de fondo MOG2 podemos deducir en base a los tiempos de ejecución que el detector MOG2 es 10 veces mas rápido que el detector Naive. Esta comparación se realiza una vez que el algoritmo de Naive toma los primeros N frames para realizar el cálculo de la mediana (es decir tenemos un "tiempo muerto" hasta que realmente funciona el código).
##### Como "talon de aquiles" de este algoritmo de detección Naive, es el necesario pre-procesamiento de los frames necesarios para calcular la mediana antes de poder realizar la máscara de "foreground". El detector de fondo Naive es 10 veces mas lento que el MOG2, por razones de calculo, aparte de tener que calcular la mediana de un nro N de frames.